In [1]:
import flask
from flask import request
import torch
import final_model

In [2]:
app = flask.Flask(__name__)
app.config["DEBUG"] = True

In [3]:
def load_model_checkpoint(path):
    checkpoint = torch.load(path)

    model = final_model.Classifier(checkpoint["input"])

    model.load_state_dict(checkpoint["state_dict"])

    return model

model = load_model_checkpoint("checkpoint.pth")

In [4]:
@app.route('/prediction', methods=['POST'])
def prediction():
    
    body = request.get_json()

    example = torch.tensor(body['data']).float()
    
    pred = model(example)
    pred = torch.exp(pred)
    _, top_class_test = pred.topk(1, dim=1)
    top_class_test = top_class_test.numpy()
    
    return {"status":"ok", "result":int(top_class_test[0][0])}

In [ ]:
app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2019 16:09:10] "POST /prediction HTTP/1.1" 200 -
